In [1]:

import numpy as np
import pandas as pd
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, BitsAndBytesConfig, pipeline, AutoModelForCausalLM
from transformers.pipelines.pt_utils import KeyDataset
from peft import PeftModel, PeftConfig
from dotenv import dotenv_values
import torch
from tqdm.auto import tqdm
import evaluate

from utils import DataPreprocessor, DatasetFormatConverter
from utils import DataPreprocessor

from src.billm import LlamaForTokenClassification

WANDB_KEY = dotenv_values(".env.base")['WANDB_KEY']
LLAMA_TOKEN = dotenv_values(".env.base")['LLAMA_TOKEN']
HF_TOKEN = dotenv_values(".env.base")['HF_TOKEN']

adapters = 'DLBDAlkemy/v3_MistralLite_adapters_4_16_32_0.05_2_0.0002'
peft_config = PeftConfig.from_pretrained(adapters)
BASE_MODEL_CHECKPOINT = peft_config.base_model_name_or_path

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_CHECKPOINT,token =HF_TOKEN)
tokenizer.pad_token = tokenizer.eos_token

/home/pferrazzi/LS-LLaMA-E3C/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
peft_config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='amazon/MistralLite', revision=None, task_type='CAUSAL_LM', inference_mode=True, r=16, target_modules={'gate_proj', 'k_proj', 'o_proj', 'q_proj', 'v_proj'}, lora_alpha=32, lora_dropout=0.05, fan_in_fan_out=False, bias='lora_only', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)

In [2]:
bnb_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_use_double_quant=True,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_compute_dtype=torch.bfloat16,
                )

base_model = AutoModelForCausalLM.from_pretrained(
    peft_config.base_model_name_or_path,
      low_cpu_mem_usage=True,
                quantization_config = bnb_config,
                return_dict=True, 
                #torch_dtype=torch.float16,
                device_map= "auto",
                cache_dir='/data/disk1/share/pferrazzi/.cache')
model = PeftModel.from_pretrained(base_model, adapters, token = HF_TOKEN)
model = model.merge_and_unload()

Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.25s/it]
You are calling `save_pretrained` to a 4-bit converted model, but your `bitsandbytes` version doesn't support it. If you want to save 4-bit models, make sure to have `bitsandbytes>=0.41.3` installed.
/home/pferrazzi/LS-LLaMA-E3C/.venv/lib/python3.11/site-packages/peft/tuners/lora/bnb.py:325: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [4]:
def prompt_maker(context, question):
    return f"<|prompter|>You are an helper which replies to the QUESTION given the CONTEXT.\n{context}\nQUESTION: {question}\n</s><|assistant|>" 

In [22]:
contesto = """Context 1: <<< 1 Project CRM Period 2015 - 2019 Project context   Objectives As the main provider of Authors' Rights services, SIAE manages incomes and distribution of royalties about the recorded works to authors and editors, whose rights are safeguarded by SIAE itself. SIAE's Customer Base consists of about 60.000 authors, editors, and other copyright holders, together with beyond 2 million various users in the last years. The Digital Transformation of SIAE in these last years ensured that customers could operate, seek assistance, or raise a complaint on a number of online channels, as well as via the traditional counters present in more than 500 branches. The Customer Base is shared among different departmental systems, with different registration regulations, data matching issues and inability to have a common vision of all the systems. The entitled customer base is registered in the Mainframe system of allocation and settlement. The whole of royalty users is registered in the Territorial Counter system, with copies in other systems as the Active Cycle and Passive Cycle systems. Online services are not linked with any departmental record. Alkemy approach   Main activities Alkemy has supported SIAE in the planning, realization, and management of a unique system of Customer Relationship Management, able to manage the customer's life cycle and trace the necessary information and all its digital trails. The implementation of such a system aims at more efficient customer management and to a correct adaptation of SIAE s services   historically a monopoly   to a modern, European market for royalty management. The Open Source solution designed by Alkemy it s based on the open source CRM vTiger, BD MySQL and ESB Mule for the coordination of registry and operational synchronizations. The CRM is integrated with the CTI CISCO system used by Contact Center and Qualified Assistance. The CRM realizes all the procedures for SIAE s Associates and Principals and provides advanced searching services for the Territorial Counter via an integrated indexing solution based on Elastic Search. Main results Only the master system in Records  management. The SIAE CRM registers about 3 million records of Copyright Holders, Users, and other customers seeking support and assistance. SIAE centralizes beyond 1000 requests of Support, Assistance, and Complaints on a daily basis.>>>
Context 2: <<<1 Contesto ed obiettivi del progetto Soluzione proposta e principali attivit  svolte AIRB system development and data environment for Basel II purposes PERIODO 2004 on-going L attivit  Nunatac per supportare il cliente nella traduzione del requisito di business si   articolata in: una fase analitica del progetto  Basilea II  che prevede il fine tuning del sistema di accettazione al fine della compliance con i requisiti regolamentari ed il successivo sviluppo dei modelli di PD ed LGD, distintamente per portafogli giovani e maturi nel caso di PD e portafogli performing e non performing nel caso di LGD; una attivit  parallela per l implementazione in ambiente SAS dell architettura funzionale del sistema di rating in cui si collocano le strutture dati di input al processo di stima di PD ed LGD e al calcolo dell esposizione; inoltre, tutti gli indicatori propedeutici al calcolo del requisito patrimoniale e tutte le informazioni necessarie per qualsiasi analisi a livello pratica. Il cliente intende implementare il sistema di rating interno dotandosi dei metodi avanzati per il calcolo del requisito patrimoniale. La specifica iniziativa diventa leva fondamentale per l introduzione di un ambiente dati SAS dedicato agli sviluppi attinenti la fase di erogazione del credito, ai processi di sviluppo produzione per l assegnazione del rating, al calcolo degli RWA e alle applicazioni che monitorano le varie componenti del rischio di credito. Il cliente ha chiesto a Nunatac lo sviluppo dell infrastruttura SAS e contemporaneamente lo sviluppo dei modelli di PD ed LGD. L ambiente SAS ospita il calcolo in produzione dei parametri di rischio ed il calcolo degli RWA.>>>
Context 3: <<<1 Retail Reporting System Bank s Retail Advanced Reporting System is designed and executed to support Branch and Head Office commercial activity using: structured data and ready KPI ready to be used; actionable reports covering sales performance, portfolio and client management, profitability; SAS In Memory data technology unique repository for data and reports accessible according to different level of need and authorization data quality and report development improvement; full reporting process completely automatized; user-friendly maintenance and development environment. New payoff design KEY FEATURES>>>.
"""
prompt = f"<|prompter|>You are an helper which replies to the QUESTION given the context. The context is: {contesto}.\n QUESTION: How can we leverage our experience in creating a unified CRM system?</s><|assistant|>"
encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
print(encodeds)
model_inputs = encodeds.to('cuda')
generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'input_ids': tensor([[32002,   995,   460,  ..., 28804,     2, 32001]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]])}
<|prompter|> You are an helper which replies to the QUESTION given the context. The context is: Context 1: <<< 1 Project CRM Period 2015 - 2019 Project context   Objectives As the main provider of Authors' Rights services, SIAE manages incomes and distribution of royalties about the recorded works to authors and editors, whose rights are safeguarded by SIAE itself. SIAE's Customer Base consists of about 60.000 authors, editors, and other copyright holders, together with beyond 2 million various users in the last years. The Digital Transformation of SIAE in these last years ensured that customers could operate, seek assistance, or raise a complaint on a number of online channels, as well as via the traditional counters present in more than 500 branches. The Customer Base is shared among different departmental systems, with different registration regulations, da

In [ ]:
question = "How can the integration of a Fraud Management system enhance the security of our e-commerce platforms?"
context = """CONTEXT 1: <<<1 Alkemy   scelta per lanciare una piattaforma e-commerce che crei un presidio di vendita diretta online su geografie prioritarie  Europa e Nord America  e conseguente governance della fase implementativa del progetto  lancio in Italia e progressivo roll-out su altre country target . Strategy digitale per il lancio di una piattaforma e-commerce Digital Experience Platform KEY ACTIVITES Definizione della strategia di brand e relativo business plan Technology Logistica Shop Management Campaign Management Web Marketing Contact Center>>>
CONTEXT 2: <<<1 Project ECOMMERCE   PIM Period 2018 - 2019 Project context   Objectives Robintur is the largest Italian network of travel agencies  over 90  and is part of the "Coop Alleanza" and "Unicoop" travel network. Alkemy was involved from Robintur for a Digital Transformation project. The main objective of the project is the "renovation" of the Robintur Travel Group website  www.robintur.it  in an e-Commerce vision-key and its integration with the Group's information systems  Legacy Systems  through a new Middleware system Alkemy approach   Main activities The plan activities for Digital Transformation, both in strategic and technological terms, are the following: Architectural Review  New Front End Systems - Middleware Introduction - interfacing with Back End Systems  Legacy Systems  ; E-Commerce  E-Commerce website redesign ; Corporate Website   Customer Area  corporate website and customer area redesign with modern features . Main results This first evolutionary step of Robintur's IT architecture provides a series of advantages: The Middleware allows to centralize and orchestrate communications between the various systems by exposing "general purpose" interfaces. The insertion and start-up of the platform also makes it possible to drastically reduce the project risks of an upgrade of the management software  from Tropico to Globo ; The Catalogue management system  PIM, Product Information Management  together with the Master Data Management  MDM  provides fundamental tools for the management of the products and for the management of the metadata that compose them; The inclusion of a modern Enterprise-class CMS equipped with E-Commerce functionality  Customer Area, acquisition funnel, etc.  allows the development of functional User Experience and modern and appealing design as well as presenting a series of additional benefits such as safety, extensibility, etc. Web marketing   eCommerce>>>
CONTEXT 3: <<<1 Fraud management system   Custom advanced reporting environment Alkemy is working on the deployment of a Fraud Management system on Cards payments for a leading international bank. This solution is based on the SAS Fraud Management platform and has to be fully integrated with Bank s core systems to respond in real-time. The proposed solution will also enable the Bank to ensure compliance with the requirements of key regulatory mandates, like Regulatory Technical Standards on Strong Customer Authentication  SCA  and Common and Secure Communication under Payment Services Directive 2  PSD2 . Fraud Management Assessment and decision making for 100  Real-time non-stop transactions 24 hours a day and 365 days a year Ready pre-defined and verified data interface Custom Advanced Reporting designed according to Bank s business needs and implemented through SAS platform embedded functionalities Off-line Merchant Profiling System KEY FEATURES>>>"""
prompt = prompt_maker(context, question)
encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
model_inputs = encodeds.to('cuda')
generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)

In [28]:
print(decoded[0])

<|prompter|> You are an helper which replies to the QUESTION given the CONTEXT.
CONTEXT 1: <<<1 Alkemy   scelta per lanciare una piattaforma e-commerce che crei un presidio di vendita diretta online su geografie prioritarie  Europa e Nord America  e conseguente governance della fase implementativa del progetto  lancio in Italia e progressivo roll-out su altre country target . Strategy digitale per il lancio di una piattaforma e-commerce Digital Experience Platform KEY ACTIVITES Definizione della strategia di brand e relativo business plan Technology Logistica Shop Management Campaign Management Web Marketing Contact Center>>>
CONTEXT 2: <<<1 Project ECOMMERCE   PIM Period 2018 - 2019 Project context   Objectives Robintur is the largest Italian network of travel agencies  over 90  and is part of the "Coop Alleanza" and "Unicoop" travel network. Alkemy was involved from Robintur for a Digital Transformation project. The main objective of the project is the "renovation" of the Robintur Tr

In [29]:
contesto = """CONTEXT 1: <<<1 Omnichannel Strategy   eCommerce fine-tuning Context and project s objectives Main activities and project output In this context, Alkemy s main activities were: Assessment of Alessi sales channels with the identification of main performance KPIs and macro-trends Benchmark with main Alessi competitors and comparable with the aim to identify the best practices within omnichannel approach Analysis of Alessi digital positioning and customer interaction model along all the customer journey and brand intimacy phases Omnichannel strategy definition with the quantification of full economical potential Individuation and estimation of specific short terms   quick win   and medium terms omnichannel initiatives to be implemented The project enabled Alessi to define an Omnichannel strategy that valued the customer journey, and strengthened the feeling of  brand intimacy  PROJECT TIMING 2015-2016 Alessi was managing multiple sale channels and its main need was to define a customer relationship strategy with the aim to make more efficient and deeper relationships between customers and brand. Alessi therefore asked Alkemy to define and develop an omnichannel strategy, to identify and estimate the main short and medium terms initiatives to be implemented Quantification of the economic impact of the initiatives Identification of the main strategic initiatives eCommerce  Omnichannel strategy eCommerce Areas>>>
CONTEXT 2: <<<1 Digital Acquisition Project context   Objectives Fidelity is one of the most important fund management companies in the world and has been operating in all international financial markets in the field of asset management for over 40 years. Fidelity's goal is to increase brand awareness online and become a Top Player in the investment   high finance sector. From the first half of 2017 Alkemy, together with Fidelity, laid the foundations for the following activities concerning a new approach of the Brand to 360   Digital Communication: Digital Assessment Digital positioning strategy Activation of display investments  Google Display Network, Gmail, Real Time Bidding, Native  that allow Fidelity to intercept new customers Activate search engine investments through Google AdWords to ensure an important position for Fidelity SEO optimization at a technical and semantic level in order to guarantee a growth of Fidelity's organic traffic Alkemy approach   Main activities Main results   14  of organic traffic compared to the previous year organic positioning on the front page of keyword brand and no-brand Web Marketing campaigns generated 70  of new sessions on the Fidelity-Italy website 775.000 clicks to the website 140 MY IMPRESSION Project PAID MEDIA Period 2017 - 2018>>>
CONTEXT 3: <<<1 eCommerce B2B Entry strategy   operational readiness Context and project s objectives Main activities and project output PROJECT TIMING 2020 During 2020 in a market context characterized by the push on digital channels, RCF evaluated the launch of a B2B eCommerce In this regard, RCF asked Alkemy to evaluate the feasibility of launching the eCommerce channel with the dual objective of generating direct sales and guaranteeing the safeguarding of the physical distribution network eCommerce Areas eCommerce Entry Strategy In this context, Alkemy developed the project with a team of strategic consultants completing the following activities:: Definition of the commercial proposal in terms of products and services for the eCommerce channel through the analysis of the RCF product offer, the digital demand for the product categories and benchmark analysis on market best practices in the reference industry and cross industry Evaluation of RCF readiness for the eCommerce operating model and design of the to-be operating model, including the design of the possible model of engagement of the physical distribution network on leads generated on the eCommerce channel, and related sourcing strategy  inhouse VS outsourcing  activities Elaboration of a detailed business plan with evaluation of the multichannel and omnichannel impacts and drafting of the implementation roadmap in the various geographies Operating model and business plan Digital demand and eCommerce products catalogue DEFINITION OF THE COMMERCIAL PROPOSAL READINESS ASSESSMENT OFORF THE eCOMMERCE MODEL BUSINESS PLAN>>>"""
question="What are the key elements to consider when defining an omnichannel strategy for a brand to enhance customer interaction and increase brand awareness online?"
prompt=prompt_maker(contesto, question)
encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
model_inputs = encodeds.to('cuda')
generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<|prompter|> You are an helper which replies to the QUESTION given the CONTEXT.
CONTEXT 1: <<<1 Omnichannel Strategy   eCommerce fine-tuning Context and project s objectives Main activities and project output In this context, Alkemy s main activities were: Assessment of Alessi sales channels with the identification of main performance KPIs and macro-trends Benchmark with main Alessi competitors and comparable with the aim to identify the best practices within omnichannel approach Analysis of Alessi digital positioning and customer interaction model along all the customer journey and brand intimacy phases Omnichannel strategy definition with the quantification of full economical potential Individuation and estimation of specific short terms   quick win   and medium terms omnichannel initiatives to be implemented The project enabled Alessi to define an Omnichannel strategy that valued the customer journey, and strengthened the feeling of  brand intimacy  PROJECT TIMING 2015-2016 Alessi w

In [6]:
question = """How can we improve the efficiency and quality control of our production and packaging lines through digital transformation?"""
context="""CONTEXT 1:<<<eCommerce Full Potential Context and project s objectives Main activities and project output PROJECT TIMING eCommerce full potential 2015 Aree eCommerce Approccio della strategia eCommerce Luxottica non   soddisfatta della performance dell'eCommerce di Sunglass Hut, che ha mostrato un basso tasso di conversione e un significativo spostamento del traffico verso i canali mobile Luxottica ha chiesto ad Alkemy di valutare le potenzialit  dell'eCommerce di Sunglass Hut, con focus specifico sulla strategia mobile Di seguito le attivit  pi  rilevanti portate a termine da Alkemy: Valutazione delle prestazioni eCommerce di Sunglass Hut tramite analisi dei KPI del sito e benchmark competitors Individuazione delle principali aree di miglioramento del sito eCommerce e individuazione delle leve strategiche chiave per l'ottimizzazione dei KPI Identificazione di iniziative specifiche e prioritizzazione in base al potenziale di beneficio economico Definizione di sinergie multicanale tra canale online e canale fisico Quantificazione degli impatti economici, sviluppo del business plan e definizione della roadmap di implementazione Le iniziative strategiche individuate sono state implementate sul portale Sunglass Hut, portando un miglioramento complessivo dei KPI di navigazione e delle vendite da desktop e mobile Quantificazione dell impatto economico delle iniziative 2>>>
CONTEXT 2: <<<eCommerce Strategy and eCommerce processes development Context and project s objectives Main activities and project output In this context, Alkemy s main activities were: Re-design of Ducati s services leveraging on digital channel, boosting the rate of brand-customers interaction Identification of brand experience purchase moment through the creation of new products and services within value chain Identification of new market segments to be addressed Development of processes and technologies to allow Ducati digital purpose  design and development of eCommerce UX UI, logistic model definition, performance activities set-up  PROJECT TIMING eCommerce Strategy, UX UI design, performance activity set-up 2017-2019 Ducati Motor Holding wanted to improve its Brans on digital channel with the aim of gathering an incresing number of customers and bigger market share Ducati asked Alkemy to develop a roadmap of strategic initiatives leading the digital development of Ducati ecosystem eCommerce Areas Design of business processes Identification of the main strategic initiatives 1>>>
CONTEXT 3: <<<1 Omnichannel Strategy   eCommerce fine-tuning Context and project s objectives Main activities and project output In this context, Alkemy s main activities were: Assessment of Alessi sales channels with the identification of main performance KPIs and macro-trends Benchmark with main Alessi competitors and comparable with the aim to identify the best practices within omnichannel approach Analysis of Alessi digital positioning and customer interaction model along all the customer journey and brand intimacy phases Omnichannel strategy definition with the quantification of full economical potential Individuation and estimation of specific short terms   quick win   and medium terms omnichannel initiatives to be implemented The project enabled Alessi to define an Omnichannel strategy that valued the customer journey, and strengthened the feeling of  brand intimacy  PROJECT TIMING 2015-2016 Alessi was managing multiple sale channels and its main need was to define a customer relationship strategy with the aim to make more efficient and deeper relationships between customers and brand. Alessi therefore asked Alkemy to define and develop an omnichannel strategy, to identify and estimate the main short and medium terms initiatives to be implemented Quantification of the economic impact of the initiatives Identification of the main strategic initiatives eCommerce  Omnichannel strategy eCommerce Areas>>>
"""
prompt=prompt_maker(context, question)
encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
model_inputs = encodeds.to('cuda')
generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<|prompter|> You are an helper which replies to the QUESTION given the CONTEXT.
CONTEXT 1:<<<eCommerce Full Potential Context and project s objectives Main activities and project output PROJECT TIMING eCommerce full potential 2015 Aree eCommerce Approccio della strategia eCommerce Luxottica non   soddisfatta della performance dell'eCommerce di Sunglass Hut, che ha mostrato un basso tasso di conversione e un significativo spostamento del traffico verso i canali mobile Luxottica ha chiesto ad Alkemy di valutare le potenzialit  dell'eCommerce di Sunglass Hut, con focus specifico sulla strategia mobile Di seguito le attivit  pi  rilevanti portate a termine da Alkemy: Valutazione delle prestazioni eCommerce di Sunglass Hut tramite analisi dei KPI del sito e benchmark competitors Individuazione delle principali aree di miglioramento del sito eCommerce e individuazione delle leve strategiche chiave per l'ottimizzazione dei KPI Identificazione di iniziative specifiche e prioritizzazione in bas

In [7]:
question = """How can we improve the efficiency and quality control of our production and packaging lines through digital transformation?"""
context="""CONTEXT: <<<1 Omnichannel Strategy   eCommerce fine-tuning Context and project s objectives Main activities and project output In this context, Alkemy s main activities were: Assessment of Alessi sales channels with the identification of main performance KPIs and macro-trends Benchmark with main Alessi competitors and comparable with the aim to identify the best practices within omnichannel approach Analysis of Alessi digital positioning and customer interaction model along all the customer journey and brand intimacy phases Omnichannel strategy definition with the quantification of full economical potential Individuation and estimation of specific short terms   quick win   and medium terms omnichannel initiatives to be implemented The project enabled Alessi to define an Omnichannel strategy that valued the customer journey, and strengthened the feeling of  brand intimacy  PROJECT TIMING 2015-2016 Alessi was managing multiple sale channels and its main need was to define a customer relationship strategy with the aim to make more efficient and deeper relationships between customers and brand. Alessi therefore asked Alkemy to define and develop an omnichannel strategy, to identify and estimate the main short and medium terms initiatives to be implemented Quantification of the economic impact of the initiatives Identification of the main strategic initiatives eCommerce  Omnichannel strategy eCommerce Areas>>>
"""
prompt=prompt_maker(context, question)
encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
model_inputs = encodeds.to('cuda')
generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<|prompter|> You are an helper which replies to the QUESTION given the CONTEXT.
CONTEXT: <<<1 Omnichannel Strategy   eCommerce fine-tuning Context and project s objectives Main activities and project output In this context, Alkemy s main activities were: Assessment of Alessi sales channels with the identification of main performance KPIs and macro-trends Benchmark with main Alessi competitors and comparable with the aim to identify the best practices within omnichannel approach Analysis of Alessi digital positioning and customer interaction model along all the customer journey and brand intimacy phases Omnichannel strategy definition with the quantification of full economical potential Individuation and estimation of specific short terms   quick win   and medium terms omnichannel initiatives to be implemented The project enabled Alessi to define an Omnichannel strategy that valued the customer journey, and strengthened the feeling of  brand intimacy  PROJECT TIMING 2015-2016 Alessi was

In [8]:
question = """How can we improve the efficiency and quality control of our production and packaging lines through digital transformation?"""
context="""CONTEXT: <<<1 Omnichannel Strategy   eCommerce fine-tuning Context and project s objectives Main activities and project output In this context, Alkemy s main activities were: Assessment of Alessi sales channels with the identification of main performance KPIs and macro-trends Benchmark with main Alessi competitors and comparable with the aim to identify the best practices within omnichannel approach Analysis of Alessi digital positioning and customer interaction model along all the customer journey and brand intimacy phases Omnichannel strategy definition with the quantification of full economical potential Individuation and estimation of specific short terms   quick win   and medium terms omnichannel initiatives to be implemented The project enabled Alessi to define an Omnichannel strategy that valued the customer journey, and strengthened the feeling of  brand intimacy  PROJECT TIMING 2015-2016 Alessi was managing multiple sale channels and its main need was to define a customer relationship strategy with the aim to make more efficient and deeper relationships between customers and brand. Alessi therefore asked Alkemy to define and develop an omnichannel strategy, to identify and estimate the main short and medium terms initiatives to be implemented Quantification of the economic impact of the initiatives Identification of the main strategic initiatives eCommerce  Omnichannel strategy eCommerce Areas>>>
"""
prompt=prompt_maker(context, question)
encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
model_inputs = encodeds.to('cuda')
generated_ids = base_model.generate(**model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<|prompter|> You are an helper which replies to the QUESTION given the CONTEXT.
CONTEXT: <<<1 Omnichannel Strategy   eCommerce fine-tuning Context and project s objectives Main activities and project output In this context, Alkemy s main activities were: Assessment of Alessi sales channels with the identification of main performance KPIs and macro-trends Benchmark with main Alessi competitors and comparable with the aim to identify the best practices within omnichannel approach Analysis of Alessi digital positioning and customer interaction model along all the customer journey and brand intimacy phases Omnichannel strategy definition with the quantification of full economical potential Individuation and estimation of specific short terms   quick win   and medium terms omnichannel initiatives to be implemented The project enabled Alessi to define an Omnichannel strategy that valued the customer journey, and strengthened the feeling of  brand intimacy  PROJECT TIMING 2015-2016 Alessi was